In [ ]:
import pandas as pd
from xlwings import Workbook, Range
import os, re
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
er_path = r"C:\Users\ReddyAl\Documents\Surveys\2016\TSG\Compensation Planning Survey\Oracle Employee Report 10-Oct-2016.xlsx"
fd_path = r"C:\Users\ReddyAl\Documents\Surveys\2016\TSG\Compensation Planning Survey\Focal 2016 12-Oct-2016 1.30 PM IST.xlsx"
mip_path = r"C:\Users\ReddyAl\Documents\Surveys\2016\TSG\Compensation Planning Survey\FY16 MIP Master File 20-Oct-2016.XLSX"

er = pd.read_excel(er_path, encoding = 'utf-8')
fd = pd.read_excel(fd_path, encoding = 'utf-8')
mip = pd.read_excel(mip_path, encoding = 'utf-8', sheetname="MIP Emp Summary & Metrics")

In [ ]:
def headerfiller(df,replace_chars = {' ':'_'
                                     ,'\n':'_'
                                     ,'-':'_'
                                     ,'%':'Perc'
                                     ,'\?':'_Que'}):
    for key, value in replace_chars.iteritems():
        df.columns = [re.sub(key, value, c) for c in df.columns]

headerfiller(er)
headerfiller(fd)
headerfiller(mip)

In [ ]:
df = er.merge(fd, left_on='PID', right_on='Employee_ID', how='inner')
df = df.merge(mip, on='PID', how ='left')

In [ ]:
df = df[df.Payroll_Country_x.isin(['United States']) & df.Payroll_Country_y.isin(['United States'])]

# Non Exempt

In [ ]:
nonexempt = df[df['FLSA_Status_Description']=='Non-Exempt']

## Non Exempt merit as percent of total

In [ ]:
x = nonexempt.Proposed_Merit_Increase_Amount.sum()
y = nonexempt.Current_Salary_FTE.sum()

z = x/y*100

print z

## Non Exempt general (Adj) as percent of total

In [ ]:
x = nonexempt[nonexempt['Additional_Increase_Reason'] == 'Adjustment'].Additional_Increase_Amount.sum()
y = nonexempt.Current_Salary_FTE .sum()

z = x/y*100

print z

## Non Exempt Promotion as percent of total

In [ ]:
x = nonexempt[nonexempt['Additional_Increase_Reason'] == 'Promotion'].Additional_Increase_Amount.sum()
y = nonexempt.Current_Salary_FTE .sum()

z = x/y*100

print z

# Exempt Non-Management

In [ ]:
exempt_nonmanage = df[(df['FLSA_Status_Description'] == 'Exempt') & 
                      df['Career_Band'].isin(['Professional', 'Business Support', 'Technical Support'])]

## Exempt Non-Management merit as percent of total

In [ ]:
x = exempt_nonmanage.Proposed_Merit_Increase_Amount.sum()
y = exempt_nonmanage.Current_Salary_FTE .sum()

z = x/y*100

print z

## Exempt Non-Management general (adj) as percent of total

In [ ]:
x = exempt_nonmanage[exempt_nonmanage['Additional_Increase_Reason'] == 'Adjustment'].Additional_Increase_Amount.sum()
y = exempt_nonmanage.Current_Salary_FTE .sum()

z = x/y*100

print z

## Exempt Non-Management promotion as percent of total

In [ ]:
x = exempt_nonmanage[exempt_nonmanage['Additional_Increase_Reason'] == 'Promotion'].Proposed_Merit_Increase_Amount.sum()
y = exempt_nonmanage.Current_Salary_FTE .sum()

z = x/y*100

print z

# Management

In [ ]:
manage = df[df['Career_Band'].isin(['Supervisory / Management'])]

## Management merit as percent of total

In [ ]:
x = manage.Proposed_Merit_Increase_Amount.sum()
y = manage.Current_Salary_FTE .sum()

z = x/y*100

print z

## Management general (adj) as percent of total

In [ ]:
x = manage[manage['Additional_Increase_Reason'] == 'Adjustment'].Additional_Increase_Amount.sum()
y = manage.Current_Salary_FTE .sum()

z = x/y*100

print z

## Management promotion as percent of total

In [ ]:
x = manage[manage['Additional_Increase_Reason'] == 'Promotion'].Additional_Increase_Amount.sum()
y = manage.Current_Salary_FTE .sum()

z = x/y*100

print z

# Executive

In [ ]:
executive = df[df['Career_Band'].isin(['Executive / Senior Management'])]

## Executive merit as percent of total

In [ ]:
x = executive.Proposed_Merit_Increase_Amount.sum()
y = executive.Current_Salary_FTE .sum()

z = x/y*100

print z

## Executive general (adj) as percent of total

In [ ]:
x = executive[executive['Additional_Increase_Reason'] == 'Adjustment'].Additional_Increase_Amount.sum()
y = executive.Current_Salary_FTE .sum()

z = x/y*100

print z

## Executive promotion as percent of total

In [ ]:
x = executive[executive['Additional_Increase_Reason'] == 'Promotion'].Additional_Increase_Amount.sum()
y = executive.Current_Salary_FTE .sum()

z = x/y*100

print z

# Over All

## Merit

In [ ]:
x = df.Proposed_Merit_Increase_Amount.sum()
y = df.Current_Salary_FTE .sum()

z = x/y*100

print z

## Adjustment

In [ ]:
x = df.Additional_Increase_Amount.sum()
y = df.Current_Salary_FTE .sum()

z = x/y*100

print z

## Promotion

In [ ]:
x = df.Additional_Increase_Amount.sum()
y = df.Current_Salary_FTE .sum()

z = x/y*100

print z

# Short-Term Incentive Plans

In [ ]:
_list = ['MIP', 'MIP2']

df.loc[(df.Bonus_Plan.isin(_list)), 'TSGBonusPlan'] = 'MIP'
df.loc[(df.Bonus_Plan_Override.isin(_list)), 'TSGBonusPlan'] = 'MIP'

In [ ]:
df.TSGBonusPlan.fillna('NoMIP', inplace=True)
df.TSGBonusPlan.value_counts(normalize=True)*100

In [ ]:
df['TSGBonusPerc'] = df[df.Bonus_Plan.isin(['MIP', 'MIP2'])]['Bonus_Percent']
df.loc[df.Bonus_Plan_Override.isin(['MIP', 'MIP2']), 'TSGBonusPerc'] = df['Bonus_Percent_Override']

In [ ]:
#Testing
print (df.TSGBonusPerc.sum())
print (df[df.Bonus_Plan.isin(['MIP','MIP2'])].Bonus_Percent.sum())
print (df[df.Bonus_Plan_Override.isin(['MIP','MIP2'])].Bonus_Percent_Override.sum())
print (df[df.Bonus_Plan.isin(['MIP','MIP2'])].Bonus_Percent.sum() + 
       df[df.Bonus_Plan_Override.isin(['MIP','MIP2'])].Bonus_Percent_Override.sum())

## % of Non-Exempt eligible to receive MIP

In [ ]:
print (df[df['FLSA_Status_Description'] == 'Non-Exempt'].TSGBonusPlan.value_counts(normalize = True)*100)

x = df[df['FLSA_Status_Description'] == 'Non-Exempt'].TSGBonusPerc.multiply(df.Annual_Salary_If_Full_time).sum()

y = df[(df['TSGBonusPlan']=='MIP') 
       & (df['FLSA_Status_Description'] == 'Non-Exempt')].Annual_Salary_If_Full_time.sum()

z = x/y*100


a = df[df['FLSA_Status_Description'] == 'Non-Exempt'].Final_Discounted_MIP_USD.sum()

b = df[(df['TSGBonusPlan']=='MIP') 
       & (df['FLSA_Status_Description'] == 'Non-Exempt')].Annual_Salary_If_Full_time.sum()

c = a/b*100


i = df[df['FLSA_Status_Description'] == 'Non-Exempt'].TSGBonusPerc.multiply(df.Final_Salary_FTE).sum()

j = df[(df['TSGBonusPlan']=='MIP') 
       & (df['FLSA_Status_Description'] == 'Non-Exempt')].Final_Salary_FTE.sum()

k = i/j*100



print ("Total Bonus Budgeted : " + str(x))
print ("Total Annual Sal of MIP Emps: " + str(y))
print ("MIP Budget as % of Annual Sal(Sum) of Eligible Emps: " + str(z))
print ("MIP Actual as % of Annual Sal(Sum) of Eligible Emps: " + str(c))
print ("MIP Next Budget as % of Annual Sal(Sum) of Eligible Emps: " + str(k))

## % of Exempt Non-Mangement eligible

In [ ]:
print(df[(df.FLSA_Status_Description.isin(['Exempt'])) &
   (df.Career_Band.isin(['Professional', 
                           'Business Support', 
                           'Technical Support']))].TSGBonusPlan.value_counts(normalize = True)*100)

print('\n #############################################################################')

x = df[(df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional',
                             'Business Support', 
                             'Technical Support'])) & 
       (df.TSGBonusPlan.isin(['MIP']))].TSGBonusPerc.multiply(df.Annual_Salary_If_Full_time).sum()

y = df[(df['TSGBonusPlan']=='MIP') &
       (df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional', 
                             'Business Support', 
                             'Technical Support']))].Annual_Salary_If_Full_time.sum()

z = x/y*100


a = df[(df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional',
                             'Business Support', 
                             'Technical Support'])) &
      (df.TSGBonusPlan.isin(['MIP']))].Final_Discounted_MIP_USD.sum()

b = df[(df['TSGBonusPlan']=='MIP') &
       (df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional', 
                             'Business Support', 
                             'Technical Support']))].Annual_Salary_If_Full_time.sum()

c = a/b*100


i = df[(df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional',
                             'Business Support', 
                             'Technical Support'])) & 
       (df.TSGBonusPlan.isin(['MIP']))].TSGBonusPerc.multiply(df.Final_Salary_FTE).sum()

j = df[(df['TSGBonusPlan']=='MIP') &
       (df['FLSA_Status_Description'] == 'Exempt') & 
       (df.Career_Band.isin(['Professional', 
                             'Business Support', 
                             'Technical Support']))].Final_Salary_FTE.sum()

k = i/j*100


print ("Total Bonus Budgeted : " + str(x))
print ("Total Annual Sal of MIP Emps: " + str(y))
print ("MIP Budget as % of Annual Sal of Eligible Emps: " + str(z))
print ("MIP Actual as % of Annual Sal of Eligible Emps: " + str(c))
print ("MIP Next Budget as % of Annual Sal of Eligible Emps: " + str(k))

## % of Management eligible for MIP

In [ ]:
print(df[(df.Career_Band.isin(['Supervisory / Management', 
                               'Customer / Client Management and Sales']))].TSGBonusPlan.value_counts(normalize = True)*100)

print ('\n')

x = df[(df.Career_Band.isin(['Supervisory / Management', 
                             'Customer / Client Management and Sales']))].TSGBonusPerc.multiply(df.Annual_Salary_If_Full_time).sum()

y = df[(df['TSGBonusPlan']=='MIP') & 
       (df.Career_Band.isin(['Supervisory / Management',
                             'Customer / Client Management and Sales']))].Annual_Salary_If_Full_time.sum()

z = x/y*100


a = df[(df.Career_Band.isin(['Supervisory / Management', 
                             'Customer / Client Management and Sales']))].Final_Discounted_MIP_USD.sum()

b = df[(df['TSGBonusPlan']=='MIP') & 
       (df.Career_Band.isin(['Supervisory / Management',
                             'Customer / Client Management and Sales']))].Annual_Salary_If_Full_time.sum()

c = a/b*100


i = df[(df.Career_Band.isin(['Supervisory / Management', 
                             'Customer / Client Management and Sales']))].TSGBonusPerc.multiply(df.Final_Salary_FTE).sum()

j = df[(df['TSGBonusPlan']=='MIP') & 
       (df.Career_Band.isin(['Supervisory / Management',
                             'Customer / Client Management and Sales']))].Final_Salary_FTE.sum()

k = i/j*100



print ("Total Bonus Budgeted : " + str(x))
print ("Total Annual Sal of MIP Emps: " + str(y))
print ("MIP Budget as % of Annual Sal of Eligible Emps: " + str(z))
print ("MIP Actual as % of Annual Sal of Eligible Emps: " + str(c))
print ("MIP Next Budget as % of Annual Sal of Eligible Emps: " + str(k))

## % of Executive eligible for MIP

In [ ]:
print (df[(df.Career_Band.isin(['Executive / Senior Management']))].TSGBonusPlan.value_counts(normalize = True)*100)

print ('\n')

x = df[(df.Career_Band.isin(['Executive / Senior Management']))].TSGBonusPerc.multiply(df.Annual_Salary_If_Full_time).sum()

y = df[(df['TSGBonusPlan']=='MIP') & 
       df.Career_Band.isin(['Executive / Senior Management'])].Annual_Salary_If_Full_time.sum()

z = x/y*100

a = df[(df.Career_Band.isin(['Executive / Senior Management']))].Final_Discounted_MIP_USD.sum()

b = df[(df['TSGBonusPlan']=='MIP') & 
       df.Career_Band.isin(['Executive / Senior Management'])].Annual_Salary_If_Full_time.sum()

c = a/b*100

i = df[(df.Career_Band.isin(['Executive / Senior Management']))].TSGBonusPerc.multiply(df.Final_Salary_FTE).sum()

j = df[(df['TSGBonusPlan']=='MIP') & 
       df.Career_Band.isin(['Executive / Senior Management'])].Final_Salary_FTE.sum()

k = i/j*100


print ("Total Bonus Budgeted : " + str(x))
print ("Total Annual Sal of MIP Emps: " + str(y))
print ("MIP Budget as % of Annual Sal of Eligible Emps: " + str(z))
print ("MIP Actual as % of Annual Sal of Eligible Emps: " + str(c))
print ("MIP Next Budget as % of Annual Sal of Eligible Emps: " + str(k))

# Average Budgeted

In [ ]:
print (df[(df.FLSA_Status_Description.isin(['Non-Exempt'])) & (df.Bonus_Plan_Override.isin(['MIP', 'MIP2']))]['Bonus_Percent_Override'])


In [ ]:
df.Career_Band.value_counts()

In [ ]:
df.FLSA_Status_Description.value_counts()

In [ ]:
df.pivot_table(values='Total_Increase_Amount', index=['FLSA_Status_Description'], columns=['Career_Band'])

In [ ]:
df[df.FLSA_Status_Description.isin(['Unknown'])][['Assignment_Category', 'Payroll_Country_x','Payroll_Country_y']]